In [68]:
import pandas as pd
import json
data = pd.read_csv('pv_week_meteo.csv')

In [69]:
data_ts_t = data[['timestamp', 'T_ambient_degC']]

In [70]:
data_ts_t.rename(columns={'timestamp': 'ds', 'T_ambient_degC': 'y'}, inplace=True)

/var/folders/kv/6z6fj5td23z74h5w23gx_d0r0000gn/T/ipykernel_58583/2021538147.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_ts_t.rename(columns={'timestamp': 'ds', 'T_ambient_degC': 'y'}, inplace=True)


In [72]:
import prophet
model = prophet.Prophet()
datas = []
import numpy as np
for i in range(5):
    cur_data = data_ts_t.copy()
    cur_data['ds'] = pd.to_datetime(cur_data['ds']) + datetime.timedelta(weeks=i)
    cur_data['y'] = cur_data['y'] + np.random.normal(0, 0.5, size=cur_data.shape[0])
    
    datas.append(cur_data)
    
data_ts_t = pd.concat(datas, ignore_index=True)


model = model.fit(data_ts_t)

10:00:05 - cmdstanpy - INFO - Chain [1] start processing
10:00:05 - cmdstanpy - INFO - Chain [1] done processing


In [74]:
from prophet.serialize import model_to_json, model_from_json

with open('serialized_model.json', 'w') as fout:
    fout.write(model_to_json(model))  # m is your Prophet mode

In [ ]:
import paho.mqtt.client as mqtt
import json
import datetime
client = mqtt.Client()
client.connect("localhost", 1883, 60)
import time

for week in range(5):
    for (time_, temp) in zip(data_ts_t['ds'], data_ts_t['y']):
        time_ = str(time_)
        payload = {
            "topic": f"{'jsi'}/{'grid'}/things/twin/commands/merge",
            "timestamp":  str(datetime.datetime.strptime(time_, "%Y-%m-%d %H:%M:%S") + datetime.timedelta(weeks=week)),
            "headers": {
                "content-type": "application/merge-patch+json"
            },
            "path": "/features",
            "value": {
                'meteo':{
                    "TdegC": temp + np.random.normal(0, 0.5)
                }
            }
        }
        json_payload = json.dumps(payload)
        client.publish("sensors/temp/live", payload= json_payload, qos=0)
        time.sleep(5)

client.disconnect() 

/var/folders/kv/6z6fj5td23z74h5w23gx_d0r0000gn/T/ipykernel_58583/3311663815.py:4: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client()
